In [3]:
# import dependencies
import pandas as pd
import numpy as np

In [4]:
# load in files

lab_file = pd.read_csv("Resources_raw/lab_data_raw.csv")
rx_file = pd.read_csv("Resources_raw/reactor_data_raw.csv")
ta_file = pd.read_csv("Resources_raw/towerA_data_raw.csv")
tb_file = pd.read_csv("Resources_raw/towerB_data_raw.csv")
tc_file = pd.read_csv("Resources_raw/towerC_data_raw.csv")

# csv to dataframe

lab_data = pd.DataFrame(lab_file)
rx_data = pd.DataFrame(rx_file)
ta_data = pd.DataFrame(ta_file)
tb_data = pd.DataFrame(tb_file)
tc_data = pd.DataFrame(tc_file)

In [22]:
def clean_data(df):
    
    # make list of df columns
    df_columns = list(df)
    
    # convert necessary objects to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # add hour column for sorting invalid checkpoints
    df['Hour'] = pd.to_datetime(df['Date']).dt.hour
    
    # keep only valid hour checkpoints
    df = df.loc[(df['Hour'] == 6) | (df['Hour'] == 13)]
    
    # store start and end dates of turnaround as variables
    start = "2018-10-12 13:00:00"
    end = "2019-03-29 6:00:00"
    
    # dates between start and end converted to null
    df.loc[(df['Date'] > start) & (df['Date'] <= end)] = np.nan
    
    # for loop to nullify text strings
    for name in df_columns:
        df.loc[df[name] == "I/O Timeout"] = np.nan
        df.loc[df[name] == "Error"] = np.nan
        df.loc[df[name] == "Bad"] = np.nan
        df.loc[df[name] == "Bad Input"] = np.nan
        df.loc[df[name] == "No Data"] = np.nan
        
    # drop nulls
    df = df.dropna()
    
    # convert all other object types to float
    df = df.astype(dict.fromkeys(df.columns[1:-1], float))
    
    # reset index
    df = df.reset_index()
    
    # drop unneeded columns (index and hour)
    df = df.drop(columns=['Hour', 'index'])
    
    return df

In [12]:
# clean lab data
lab_data = clean_data(lab_data)

C:\Users\anna\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [18]:
lab_data.describe()

,TA.OVH.RECYCLE.NC4.VOLPCT
count,5342.000000
mean,11.136153
std,14.474349
min,0.000000
25%,8.200000
50%,10.300000
75%,12.800000
max,1005.000000


In [19]:
# clean reactor data
rx_data = clean_data(rx_data)
rx_data.describe()

,PREFRAC RECYCLE TO CONT,PREFRAC RECYCLE TO CONT.1,ALKY NORTH CONTACT. FEED,ALKY NORTH CONTACT. FEED.1,NORTH CONTACTOR FLUSH,NORTH RX OUTLET TEMP,DEISO RECYCLE TO CONTACT,DEISO RECYCLE TO CONTACT.1,ALKY SOUTH CONTACT. FEED,ALKY SOUTH CONTACT. FEED.1,SOUTH CONTACTOR FLUSH,SOUTH RX OUTLET TEMP,SOUTH CONTACT C.W.R.,CONDENSATE PH MONITOR,CONTACTOR DELTA T CONTRL
count,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000
mean,2761.123859,28.004040,2685.611161,23.140521,131.203216,91.476504,6768.491359,55.275676,2068.195815,18.320759,131.427272,92.292182,78.365939,7.209025,-0.797493
std,1097.880255,10.762031,550.593059,8.099408,13.792705,7.731235,1020.313107,4.901060,437.958774,5.009712,20.062681,7.517265,7.345467,1.884970,3.335101
min,0.000000,-6.900000,0.000000,-6.900001,0.000000,39.115906,0.000000,-6.900001,0.000000,-6.900001,0.000000,53.540039,51.538086,-0.089193,-47.111736
25%,2046.409607,18.000000,2416.367248,18.177530,124.570370,86.833942,6314.628174,53.140571,1864.641632,16.280122,118.794918,87.878038,74.310684,6.520175,-2.876673
50%,2635.391357,27.163007,2678.655395,21.217908,127.410889,92.583805,6508.248535,55.221201,2104.441407,18.652915,124.353680,93.508320,80.143162,7.547871,-0.276517
75%,3465.538757,35.668804,2992.078308,29.855751,133.004734,97.124060,6843.628052,57.215943,2343.136658,21.153531,141.758537,97.777939,83.795166,8.324176,0.891529
max,7763.305664,105.000000,4402.867188,100.000000,252.685547,110.000893,16473.312500,68.686150,3701.582275,45.000000,252.983093,109.204102,93.135925,14.439209,46.707916


In [20]:
# clean Tower A data
ta_data = clean_data(ta_data)
ta_data.describe()

,TA.Charge.BPD,TA.OH.PSIG,TA.Tray4.F,TA.OH.F,TA.RECV.PCT,TA.RECV.PSIG,TA.REFLUX.BPD,TA.RECYCLE.TO.CONTACT.BPD,TA.RECYCLE.TO.CONTACT.F,TA.TRAY39.F,TA.TRAY45.F,TA.DP.PSIG,TA.BTM.PSIG,TA.BTM.LEVEL.PCT,TA.BTM.REBOIL.OUT.F,TA.TOP.REBOIL.OUT.F
count,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000
mean,15624.428569,97.724862,134.155713,131.522996,9.290074,100.181961,6992.693161,6726.644595,84.438646,164.226270,178.433037,7.278620,103.291009,90.847109,309.604144,210.518725
std,4264.826486,6.224325,3.587819,4.000799,9.003224,4.538600,1307.059555,989.960174,8.754896,6.900231,10.476421,9.096112,10.362998,12.979410,12.242742,17.863358
min,0.000000,-1.532560,102.691650,102.038574,-6.900000,57.708740,-836.645019,0.000000,27.513123,131.103516,136.929321,0.002177,3.174019,0.000000,185.351562,150.372314
25%,12189.267090,96.931414,132.427978,129.534149,6.732341,99.589539,6351.218872,6297.223755,79.359818,160.897350,174.343872,5.144447,99.865723,82.110928,301.676941,197.711182
50%,15698.013670,97.401123,133.938599,131.687927,9.928573,100.012207,7143.522216,6497.671875,85.122681,164.300552,177.410904,7.573607,104.711151,87.989479,310.391235,207.223518
75%,17979.037597,99.453428,135.274635,133.263462,13.606769,100.360107,7898.555787,6777.425781,89.757053,167.592548,181.070332,8.386230,106.025507,102.352913,319.514465,219.818130
max,29810.134770,133.076477,194.899170,194.660858,105.000000,136.087036,12522.957030,16473.312500,113.681038,299.072266,331.774933,101.661385,141.533661,199.996948,359.384460,329.602081


In [23]:
tb_data = clean_data(tb_data)
tb_data.describe()

C:\Users\anna\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,TB.CHARGE.BPD,TB.CHARGE.PREHEAT.COND.LBHR,TB.CHARGE.F,TB.OVHD.F,TB.OVHD.PSIG,TB.OVHD.REVEIVER.PSIG,TB.OVHD.REVEIVER.F,TB.OVHD.REVEIVER.LVL.PCT,TB.BTM.PSIG,TB.BTM.LVL.PCT,TB.DP.PSIG,TB.BTM.F,TB.REBOIL.OUT.F
count,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000
mean,12772.576421,3647.188501,145.618049,175.237906,183.310000,184.814844,75.782512,61.112795,186.900032,82.212531,3.593235,213.325288,216.348596
std,1064.448754,787.050647,7.450779,5.093775,2.388552,3.063238,19.327296,46.662511,4.035574,4.265610,3.131000,5.145277,4.903160
min,5858.637207,914.653503,123.025520,154.954483,157.860306,155.394135,19.893646,-91.756966,155.940903,63.886410,-2.833572,196.509537,199.824417
25%,12141.566892,3093.009827,138.095253,172.446308,182.816875,183.020649,62.677482,40.789262,184.409168,79.995989,1.729459,209.914875,213.086197
50%,12822.943845,3731.380493,145.350647,175.749786,183.068848,184.525085,77.493286,45.076784,186.549385,84.102146,3.474136,212.543175,216.151367
75%,13393.766843,4221.606689,152.993515,178.372959,183.994247,185.989403,89.450455,67.708408,188.581070,85.054365,4.949290,217.817627,220.166130
max,16790.464840,6898.158691,185.386932,210.779907,197.756378,200.649857,137.485886,199.990845,218.297592,110.159851,27.846790,248.548935,257.684631


In [25]:
tc_data = clean_data(tc_data)
tc_data.describe()

,TC.TRAY7.F,TC.TRAY40.F,TC.CHARGE.F,TC.OVHD.PSIG,TC.OVHD.RECVR.F,TC.TRAY24.F,TC.DP.PSIG,TC.BTM.PSIG,TC.REBOIL.OUT.F,TC.REBOIL.COND.LBHR
count,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000
mean,192.221469,204.439332,166.386492,238.913951,171.965331,188.931519,3.278840,242.184423,206.660325,7025.295286
std,4.610535,4.305985,15.046051,9.563152,8.414807,13.632097,0.514274,9.620863,4.051974,1058.005927
min,86.463882,97.879448,37.571716,58.603485,74.485779,84.071915,0.017766,58.572769,98.681641,50.172615
25%,191.162556,203.845978,158.068848,236.864265,167.680370,173.988453,3.032314,240.154251,205.786133,6343.553223
50%,192.697708,204.798286,164.086914,239.973762,172.128960,195.822716,3.288918,243.133560,206.711647,7005.793213
75%,193.999901,205.913528,170.924381,242.884811,177.360535,197.891464,3.519506,246.192932,207.887211,7751.950928
max,206.664978,217.206314,209.633942,271.525848,195.043945,254.974380,10.459358,273.582672,277.923584,11856.028320
